In [1]:
!pip install --quiet langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [2]:
import os

os.environ["OPENAI_API_TYPE"] = "azure_ad"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "This form the Azure portal - model deployment"
# os.environ["AZURE_OPENAI_API_KEY"] = 'This form the Azure portal - model deployment'
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://mobility-sage-openai-services.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = '9bbf92a17be24c66881eaa5a1c6aa77d'

In [8]:
from langchain_openai import AzureChatOpenAI
from langchain.chains.router import MultiPromptChain
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
llm = AzureChatOpenAI(
            temperature=1,
            azure_deployment="gpt-4o",
        )

# Router Chain

A good way to imagine this is if you have multiple subchains, each of which is specialized for a particular type of input, you could have a router chain, which first decides which subchain to pass it to and then passes it to that chain.

In simple terms if we want out prompt to wear multiple hats or assume differnt roles based on the user input, we can use the router chain.

In [6]:
# Create multiple prompts

physics_template = """
  You are a very smart physics professor. \
  You are great at answering questions about physics in a concise and easy to understand manner. \
  When you don't know the answer to a question you admit that you don't know.

  Here is a question:
  {input}
"""


math_template = """
  You are a very good mathematician. You are great at answering math questions. \
  You are so good because you are able to break down hard problems into their component parts, \
  answer the component parts, and then put them together to answer the broader question.

  Here is a question:
  {input}
"""

In [7]:
# Build a prompt descriptor

prompt_infos = [
    {
        "name" : "Physics",
        "description" : "Good for answering question about physics",
        "prompt_template" : physics_template
    },
    {
        "name" : "Math",
        "description" : "Good for answering math questions",
        "prompt_template" : math_template

    }
]

## Build the Destination chains.

## These are the chains that will be called by the router chain.

In [44]:
destination_chain = {}

for p_info in prompt_infos:
  name = p_info['name']
  prompt_template = p_info['prompt_template']

  prompt = PromptTemplate(
      template=prompt_template,
      input_variables=['input']
  )

  chain = LLMChain(
      llm = llm,
      prompt = prompt
  )

  destination_chain[name] = chain



In [45]:
default_chain = ConversationChain(
    llm = llm,
    output_key = "text"
)

In [46]:
default_prompt = PromptTemplate.from_template("{blackhole}")
default_prompt

PromptTemplate(input_variables=['blackhole'], template='{blackhole}')

In [47]:
destinations = [f"{p['name']} : {p['description']}" for p in prompt_infos]
destinations

['Physics : Good for answering question about physics',
 'Math : Good for answering math questions']

In [48]:
destinations_str = '\n'.join(destinations)
destinations_str

'Physics : Good for answering question about physics\nMath : Good for answering math questions'

# Router Chain.

In [49]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [50]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinations_str
)

print(router_template) # built in  MULTI_PROMPT_ROUTER_TEMPLATE from langchain

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Physics : Good for answering question about physics
Math : Good for answering math questions

<< INPU

In [51]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

In [52]:
router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt
)

In [54]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chain,
    default_chain=default_chain,
    verbose=True
)

In [55]:
print(chain.run("What is black body radiation?"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Thank you for the kind words! I'd be happy to help break down the concept of black body radiation.

### Understanding Black Body Radiation:

#### 1. **Black Body Definition:**
A black body is an idealized physical object that absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence. Because it absorbs all light, it appears black when it is at a lower temperature.

#### 2. **Emission of Radiation:**
Although a black body absorbs all radiation, it also emits radiation when it is at a constant temperature. This emitted radiation is termed "black body radiation."

#### 3. **Spectral Distribution:**
The radiation emitted by a black body is a function of temperature. The distribution of wavelengths (spectra) of the emitted radiation varies with temperature in a predictable way. Several key laws describe this distribution:

   - **Planck's Law:** This law provides a formula for the intensity of radiation emitted by a black body as a function of wavelength 

In [56]:
print(chain.run("What is the first prime number greater than 40 such that one plus the prime number is divisible by 3"))

To determine the first prime number greater than 40 such that one plus the prime number is divisible by 3, we can follow these steps:

### Step-by-step Breakdown

1. **Find Prime Numbers Greater than 40:**
   We need to identify prime numbers that are greater than 40. 

   Prime numbers are numbers greater than 1 that have no positive divisors other than 1 and themselves.

2. **Check if One Plus the Prime Number is Divisible by 3:**
   We need to check if adding 1 to each prime number results in a number that is divisible by 3.

   Let \( p \) be the prime number. The condition is:
   \[
   p + 1 \equiv 0 \pmod{3}
   \]
   This simplifies to:
   \[
   p \equiv -1 \pmod{3} \quad \text{or} \quad p \equiv 2 \pmod{3}
   \]

### List of Prime Numbers Greater than 40

The first few prime numbers greater than 40 are:
\[
41, 43, 47, 53, 59, 61, 67, 71, 73, \ldots
\]

### Checking Each Prime Number

- \( 41 + 1 = 42 \) and \( 42 \) is divisible by 3 because \( 42 \div 3 = 14 \).
- \( 43 + 1 = 4

In [57]:
print(chain.run("What is the name of the type of cloud that rins"))

Thank you for your kind words! To address your question, it appears to be about identifying the type of cloud associated with rain.

The type of cloud most commonly associated with rain is called a **"nimbostratus"** cloud. These clouds are typically dark, thick, and cover the sky uniformly, often bringing continuous, steady precipitation.

To break this down:
1. **Nimbostratus Clouds** - The prefix "nimbo-" means "rain," and "stratus" refers to a sheet-like or layered form. So, together, "nimbostratus" identifies a rain-bearing cloud that appears as a widespread, thick, and gray sheet.
   
I hope this clarifies your question! If you have any more questions about clouds or any other topic, feel free to ask.
